# Collect GDELT events

## Requirements

In [ ]:
!pip install --user gdelt

In [6]:
from datetime import date, timedelta 
from gdelt import gdelt as gdelt_client
import os
import tempfile

## Getting GDELT events

In [30]:
def get_events(event_date):
    version_date = date(2015, 2, 18)
    if event_date < version_date:
        client = gdelt_client(version=1)
    else:
        client = gdelt_client(version=2)
    events = client.Search(event_date.strftime("%Y %m %d"), table="events", coverage=True)
    del client
    return events

## Filter events by country code (FIPS-4)

In [35]:
def filter_events(country_code, event_date=date.today()):
    '''FIPS-4 country code'''
    events = get_events(event_date)
    return events[events.ActionGeo_CountryCode == country_code]

def save_events(country_code, start_date=date.today(), max_days=365, out_dir=tempfile.gettempdir()):
    file_path = os.path.join(out_dir, "{0}_{1}.csv".format(country_code, start_date.strftime("%Y_%m_%d")))
    filter_date = start_date
    has_events = False
    for day_count in range(0, max_days):
        try:
            events = filter_events(country_code, filter_date)
            print(filter_date)
            if not events.empty:
                if not os.path.exists(file_path):
                    events.to_csv(file_path, index=False)
                    has_events = True
                else:
                    events.to_csv(file_path, index=False, header=False, mode="a")
        except ValueError as err:
            print(err)
        filter_date -= timedelta(days=1)
    
    if has_events:
        print(file_path)
    else:
        print("No events obtained.")
    
def wayback_events(country_code, start_date=date.today(), out_dir=tempfile.gettempdir()):
    min_date = date(1979, 1, 1)
    if start_date < min_date:
        raise ValueError("GDELT only supports 'Jan 01 1979 - Present' queries currently. Try another date!")
    
    date_diff = start_date - min_date
    max_days = date_diff.days + 1
    save_events(country_code, start_date, max_days, out_dir)

country_code = "AJ"
#start_date = date.today()
start_date = date(1990, 1, 1)
filter_events(country_code, start_date)
#save_events(country_code, start_date, 1)
#wayback_events(country_code, start_date)

1990-01-01
C:\Users\DEVELO~1\AppData\Local\Temp\AJ_1990_01_01.csv
